In [71]:
require('nn')
require('hdf5')

f = hdf5.open("data_glove_7.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
Y_valid = f:read("valid_output"):all()
X_test = f:read("test_input"):all()

nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]

f:close()

window_size = X_train:size(2)/2
print(window_size)
X_train_word = X_train:sub(1,X_train:size(1),1,window_size)
X_valid_word = X_valid:sub(1,X_valid:size(1),1,window_size)
X_train_cap = X_train:sub(1,X_train:size(1),window_size+1,2*window_size)
X_test_word = X_test:sub(1,X_test:size(1),1,window_size)

7	


In [73]:
X = X_train_word:clone()
X_v = X_valid_word:clone()
X_t = X_test_word:clone()

for i = 1,X_train_word:size(1) do
    for j =2,X_train_word:size(2) do
        X[{i,j}] = X_train_word[{i,j}] + nfeatures*(j-1)
    end
end

for i = 1,X_valid_word:size(1) do
    for j =2,X_valid_word:size(2) do
        X_v[{i,j}] = X_valid_word[{i,j}] + nfeatures*(j-1)
    end
end

for i = 1,X_test_word:size(1) do
    for j =2,X_test_word:size(2) do
        X_t[{i,j}] = X_test_word[{i,j}] + nfeatures*(j-1)
    end
end

f2 = io.open("lr_loss_7.txt", "w")

In [74]:
X_ = X
y = Y_train

addB = nn.Add(nclasses)
lookup = nn.LookupTable(nfeatures*window_size,nclasses)
sum = nn.Sum(2)
softmax = nn.LogSoftMax()
h = nn.Sequential()
h:add(lookup)
h:add(sum)
h:add(addB)
mse = nn.CrossEntropyCriterion()

In [79]:
batch = 1000
eta = 100
nEpochs = 6000

for i = 1, nEpochs do
    
    
    bsize= batch
    local idx = torch.randperm(X_:size(1)):sub(1,bsize)

    x = torch.Tensor(bsize, X_:size(2))

    y = y:reshape(y:size(1), 1)

    y_ = torch.Tensor(bsize, 1)

    for i=1,bsize do
        x[i] = X_[idx[i]]
        y_[i] = y[idx[i]]
    end

    y_ = y_:squeeze()
    
    
    -- zero out our gradients
    h:zeroGradParameters()
    -- do forward pass
    preds = h:forward(x)
    -- get loss
    loss = mse:forward(preds, y_)
    if i % 100 == 0 then
        print("epoch " .. i .. ", loss: " .. loss)
    end
    f2:write(loss .. "\n")
    if loss == 0 then break end    
    -- backprop
    dLdpreds = mse:backward(preds, y_) -- gradients of loss wrt preds
    dLdparams = h:backward(x, dLdpreds)
    h:updateParameters(eta)
end

epoch 100, loss: 0.064285654845778	


epoch 200, loss: 0.057719725177212	


epoch 300, loss: 0.055043515613919	


epoch 400, loss: 0.067976835812691	


epoch 500, loss: 0.061825208025942	


...

epoch 5600, loss: 0.050756240579658	


epoch 5700, loss: 0.065735079224052	


epoch 5800, loss: 0.051663668331273	


epoch 5900, loss: 0.062227071447145	


epoch 6000, loss: 0.049453536933094	


In [69]:
function score(input, output)
    local pred = torch.zeros(input:size(1))
    local score = torch.zeros(input:size(1))
    vec = h:forward(input)
    for i = 1,pred:size(1) do
        local max, ind = (vec[i]):max(1)
        pred[i] = ind
        if output[i] == pred[i] then
            score[i] = 1
        else
            score[i] = 0
        end
    end
    return torch.mean(score)
end

In [81]:
--print(score(X_v, Y_valid))
f2:close()

true	
